In [0]:
! /opt/bin/nvidia-smi

Thu Jan 16 23:22:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib
import time
import gensim
import pickle, json
%matplotlib inline
from matplotlib import pyplot as plt
from collections import Counter

In [0]:
training_path = '/content/drive/My Drive/Colab Notebooks/project2/train.pkl'
validation_path = '/content/drive/My Drive/Colab Notebooks/project2/validation.pkl'
word2vec_path = '/content/drive/My Drive/Colab Notebooks/project2/cn.cbow.bin'
history_path = '/content/drive/My Drive/Colab Notebooks/project2/model/history_%s.json'
model_path = '/content/drive/My Drive/Colab Notebooks/project2/model/model_%s.tf'

to_categorical = {
  -2: np.array([1., 0., 0., 0.], dtype=np.float32),
  -1: np.array([0., 1., 0., 0.], dtype=np.float32),
  0: np.array([0., 0., 1., 0.], dtype=np.float32),
  1: np.array([0., 0., 0., 1.], dtype=np.float32),
}

In [0]:
train = pickle.load(open(training_path, 'rb'))
validation = pickle.load(open(validation_path, 'rb'))

In [0]:
columns = ['location_traffic_convenience',
   'location_distance_from_business_district', 'location_easy_to_find',
   'service_wait_time', 'service_waiters_attitude',
   'service_parking_convenience', 'service_serving_speed', 'price_level',
   'price_cost_effective', 'price_discount', 'environment_decoration',
   'environment_noise', 'environment_space', 'environment_cleaness',
   'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
   'others_overall_experience', 'others_willing_to_consume_again']

train_ground_truth = {k: train[k].to_list() for k in columns}
validation_ground_truth = {k: validation[k].to_list() for k in columns}
output = columns[0]

batch_size = 8
truncate = 450
dimension = 300

In [0]:
train = train[['content', output]]
train.columns = ['x', 'y']

In [0]:
validation = validation[['content', output]]
validation.columns = ['x', 'y']

In [0]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
x_train = train['x'].to_list()
x_validation = validation['x'].to_list()

In [0]:
x_train = [word2vec.wv[el]  for el in x_train]
x_validation = [word2vec.wv[el]  for el in x_validation]
del word2vec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
class CIFAR10Sequence(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size, class_weight=None):
      assert len(x) == len(y), '输入输出长度不匹配'

      self.x = x
      # self.y = tf.keras.utils.to_categorical(y, 4)
      self.y = [to_categorical[el] for el in y]
      self.batch_size = batch_size
      self.add = [np.zeros(300, dtype=np.float32)]
      self.truncate = 450
      self.dimension = 300
      self.sample_weight = None
      if class_weight:
        std = np.std([el for el in class_weight.values()])
        class_weight = {k: v / std for k, v in class_weight.items()}
        self.sample_weight = np.array([class_weight[el] for el in y])

    def __len__(self):
      return len(self.x) // self.batch_size

    def __getitem__(self, idx):
      batch_x = self.x[idx * self.batch_size: (idx + 1) * self.batch_size]
      batch_x = [np.append(el, self.add * (self.truncate - el.shape[0]), axis=0) if el.shape[0] < self.truncate else el for el in batch_x]
      batch_y = self.y[idx * self.batch_size: (idx + 1) * self.batch_size]

      if self.sample_weight is None:
        return np.array(batch_x), np.array(batch_y)

      sample_weight = self.sample_weight[idx * self.batch_size: (idx + 1) * self.batch_size]
      return np.array(batch_x), np.array(batch_y), np.array(sample_weight)


      # batch_x = tf.convert_to_tensor(batch_x, dtype=tf.float32)
      # batch_y = tf.convert_to_tensor(batch_y, dtype=tf.float32)
      # return batch_x, batch_y


In [0]:
def make_model():
  activation = 'relu'
  activation2 = 'relu'
  dropout = 0.5

  model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(filters=16, kernel_size=3, strides=1, input_shape=(truncate, dimension)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(16),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation2),
      tf.keras.layers.Dropout(dropout),

      tf.keras.layers.Dense(8),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation2),
      tf.keras.layers.Dropout(dropout),

      tf.keras.layers.Dense(4),
      tf.keras.layers.Softmax(),
  ])
  
  model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
                loss='categorical_crossentropy',
                metrics = ['categorical_accuracy', 'AUC', 'Precision', 'Recall'])
  return model

In [0]:
# columns = ['location_traffic_convenience',
#    'location_distance_from_business_district', 'location_easy_to_find',
#    'service_wait_time', 'service_waiters_attitude',
#    'service_parking_convenience', 'service_serving_speed', 'price_level',
#    'price_cost_effective', 'price_discount', 'environment_decoration',
#    'environment_noise', 'environment_space', 'environment_cleaness',
#    'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
#    'others_overall_experience', 'others_willing_to_consume_again']

# finished_field = 'dish_recommendation '.strip()
# columns = columns[columns.index(finished_field) + 1:] if finished_field else columns

# epochs = 10

# for index, key in enumerate(columns):
#   print(key)

#   model = make_model()
  
#   y_train = train_ground_truth[key]
#   y_validation = validation_ground_truth[key]

#   class_weight = {k: len(y_validation) / v for k, v in Counter(y_validation).items()}
  
#   train = CIFAR10Sequence(x_train, y_train, batch_size, class_weight)
#   validation = CIFAR10Sequence(x_validation, y_validation, batch_size)

#   H = model.fit_generator(train, steps_per_epoch=len(train), epochs=epochs, validation_data=validation, validation_steps=len(validation), verbose=2)
#   model.save(model_path % key)
#   with open(history_path % key, 'w') as f:
#     f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
#   print(f'{key} model saved\n############################################################################################################################################################')

others_overall_experience
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/10
Epoch 1/10
13125/13125 - 236s - loss: 0.2676 - categorical_accuracy: 0.3410 - auc: 0.5995 - precision: 0.4764 - recall: 0.0434 - val_loss: 1.1384 - val_categorical_accuracy: 0.6851 - val_auc: 0.7839 - val_precision: 0.8607 - val_recall: 0.0910
Epoch 2/10
Epoch 1/10
13125/13125 - 228s - loss: 0.2463 - categorical_accuracy: 0.4475 - auc: 0.6884 - precision: 0.6107 - recall: 0.0783 - val_loss: 1.2010 - val_categorical_accuracy: 0.5789 - val_auc: 0.7278 - val_precision: 0.6648 - val_recall: 0.0947
Epoch 3/10
Epoch 1/10
13125/13125 - 229s - loss: 0.2409 - categorical_accuracy: 0.4390 - auc: 0.7029 - precision: 0.6569 - recall: 0.1015 - val_loss: 1.0935 - val_categorical_accuracy: 0.6733 - val_auc: 0.7878 - val_precision: 0.7652 - val_recall: 0.0787
Epoch 4/10
Epoch 1/10
13125/13125 - 228s - loss: 0.2372 - categorical_accuracy: 0.4157 - auc: 0.7088 - precision: 0.6154 - recal

In [0]:
# 重新训练

columns = ['others_overall_experience', ]

epochs = 10

for index, key in enumerate(columns):
  print(key)

  model = make_model()
  
  y_train = train_ground_truth[key]
  y_validation = validation_ground_truth[key]

  class_weight = {k: len(y_validation) / v for k, v in Counter(y_validation).items()}
  
  train = CIFAR10Sequence(x_train, y_train, batch_size, class_weight)
  validation = CIFAR10Sequence(x_validation, y_validation, batch_size)

  H = model.fit_generator(train, steps_per_epoch=len(train), epochs=epochs, validation_data=validation, validation_steps=len(validation), verbose=2)
  model.save(model_path % key)
  with open(history_path % key, 'w') as f:
    f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
  print(f'{key} model saved\n############################################################################################################################################################')
problem_columns = ['location_distance_from_business_district']

others_overall_experience
Epoch 1/10
Epoch 1/10
13125/13125 - 234s - loss: 0.2667 - categorical_accuracy: 0.2850 - auc: 0.5993 - precision: 0.4244 - recall: 0.0421 - val_loss: 1.1795 - val_categorical_accuracy: 0.5063 - val_auc: 0.7450 - val_precision: 0.7749 - val_recall: 0.0535
Epoch 2/10
Epoch 1/10
13125/13125 - 221s - loss: 0.2473 - categorical_accuracy: 0.3782 - auc: 0.6657 - precision: 0.6022 - recall: 0.0772 - val_loss: 1.1556 - val_categorical_accuracy: 0.4611 - val_auc: 0.7497 - val_precision: 0.6566 - val_recall: 0.0509
Epoch 3/10
Epoch 1/10
13125/13125 - 220s - loss: 0.2404 - categorical_accuracy: 0.3928 - auc: 0.6913 - precision: 0.6039 - recall: 0.0914 - val_loss: 1.2276 - val_categorical_accuracy: 0.1079 - val_auc: 0.6673 - val_precision: 0.6022 - val_recall: 0.0552
Epoch 4/10
Epoch 1/10
13125/13125 - 223s - loss: 0.2375 - categorical_accuracy: 0.3760 - auc: 0.6962 - precision: 0.6110 - recall: 0.1019 - val_loss: 1.1695 - val_categorical_accuracy: 0.2889 - val_auc: 0.7180

In [0]:
# # 再训练几个epoch

# columns = ['environment_decoration']

# finished_field = ''.strip()
# columns = columns[columns.index(finished_field) + 1:] if finished_field else columns

# epochs = 5

# for index, key in enumerate(columns):
#   model = tf.keras.models.load_model(model_path % key)

#   y_train = train_ground_truth[key]
#   y_validation = validation_ground_truth[key]

#   class_weight = {k: len(y_validation) / v * 15 for k, v in Counter(y_validation).items()}
  
#   train = CIFAR10Sequence(x_train, y_train, batch_size, class_weight)
#   validation = CIFAR10Sequence(x_validation, y_validation, batch_size)

#   H = model.fit_generator(train, steps_per_epoch=len(train), epochs=epochs, validation_data=validation, validation_steps=len(validation), verbose=2)
#   model.save(model_path % key)
#   with open(history_path % key, 'a') as f:
#     f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
#   print(f'#####################################################\n{key} model saved')

In [0]:
# 用小的lr再训练

columns = ['location_traffic_convenience',
   'location_distance_from_business_district', 'location_easy_to_find',
   'service_wait_time', 'service_waiters_attitude',
   'service_parking_convenience', 'service_serving_speed', 'price_level',
   'price_cost_effective', 'price_discount', 'environment_decoration',
   'environment_noise', 'environment_space', 'environment_cleaness',
   'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
   'others_overall_experience', 'others_willing_to_consume_again']

finished_field = ' '.strip()
columns = columns[columns.index(finished_field) + 1:] if finished_field else columns

epochs = 5
lr = 0.003

for index, key in enumerate(columns):
  model = tf.keras.models.load_model(model_path % key)
  model.compile(optimizer=tf.keras.optimizers.SGD(lr),
                loss='categorical_crossentropy',
                metrics = ['categorical_accuracy', 'AUC', 'Precision', 'Recall'])

  y_train = train_ground_truth[key]
  y_validation = validation_ground_truth[key]

  class_weight = {k: len(y_validation) / v * 15 for k, v in Counter(y_validation).items()}
  
  train = CIFAR10Sequence(x_train, y_train, batch_size, class_weight)
  validation = CIFAR10Sequence(x_validation, y_validation, batch_size)

  H = model.fit_generator(train, steps_per_epoch=len(train), epochs=epochs, validation_data=validation, validation_steps=len(validation), verbose=2)
  model.save(model_path % key)
  with open(history_path % key, 'a') as f:
    f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
  print(f'#####################################################\n{key} model saved')

Epoch 1/5
Epoch 1/5
13125/13125 - 240s - loss: 0.0946 - categorical_accuracy: 0.6511 - auc_44: 0.8857 - precision_44: 0.9846 - recall_44: 0.5585 - val_loss: 0.5934 - val_categorical_accuracy: 0.6939 - val_auc_44: 0.9368 - val_precision_44: 0.9828 - val_recall_44: 0.6498
Epoch 2/5
Epoch 1/5
13125/13125 - 233s - loss: 0.0941 - categorical_accuracy: 0.6583 - auc_44: 0.8878 - precision_44: 0.9848 - recall_44: 0.5630 - val_loss: 0.5804 - val_categorical_accuracy: 0.6957 - val_auc_44: 0.9300 - val_precision_44: 0.9818 - val_recall_44: 0.6623
Epoch 3/5
Epoch 1/5
13125/13125 - 231s - loss: 0.0938 - categorical_accuracy: 0.6416 - auc_44: 0.8859 - precision_44: 0.9851 - recall_44: 0.5671 - val_loss: 0.6077 - val_categorical_accuracy: 0.6853 - val_auc_44: 0.9216 - val_precision_44: 0.9832 - val_recall_44: 0.6466
Epoch 4/5
Epoch 1/5
13125/13125 - 230s - loss: 0.0935 - categorical_accuracy: 0.6472 - auc_44: 0.8872 - precision_44: 0.9842 - recall_44: 0.5701 - val_loss: 0.5772 - val_categorical_accur

KeyboardInterrupt: ignored

In [0]:
# 用更小的lr再训练

columns = ['location_traffic_convenience',
   'location_distance_from_business_district', 'location_easy_to_find',
   'service_wait_time', 'service_waiters_attitude',
   'service_parking_convenience', 'service_serving_speed', 'price_level',
   'price_cost_effective', 'price_discount', 'environment_decoration',
   'environment_noise', 'environment_space', 'environment_cleaness',
   'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
   'others_overall_experience', 'others_willing_to_consume_again']

finished_field = ''.strip()
columns = columns[columns.index(finished_field) + 1:] if finished_field else columns

epochs = 5
lr = 0.0005

for index, key in enumerate(columns):
  model = tf.keras.models.load_model(model_path % key)
  model.compile(optimizer=tf.keras.optimizers.SGD(lr),
                loss='categorical_crossentropy',
                metrics = ['categorical_accuracy', 'AUC', 'Precision', 'Recall'])

  y_train = train_ground_truth[key]
  y_validation = validation_ground_truth[key]
  class_weight = {k: len(y_validation) / v * 15 for k, v in Counter(y_validation).items()}
  
  train = CIFAR10Sequence(x_train, y_train, batch_size, class_weight)
  validation = CIFAR10Sequence(x_validation, y_validation, batch_size)

  H = model.fit_generator(train, steps_per_epoch=len(train), epochs=epochs, validation_data=validation, validation_steps=len(validation), verbose=2)
  model.save(model_path % key)
  with open(history_path % key, 'a') as f:
    f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
  print(f'#####################################################\n{key} model saved')

Epoch 1/5
Epoch 1/5
13125/13125 - 229s - loss: 0.0949 - categorical_accuracy: 0.6458 - auc_32: 0.8831 - precision_32: 0.9841 - recall_32: 0.5512 - val_loss: 0.5698 - val_categorical_accuracy: 0.7006 - val_auc_32: 0.9381 - val_precision_32: 0.9817 - val_recall_32: 0.6612
Epoch 2/5
Epoch 1/5
13125/13125 - 225s - loss: 0.0947 - categorical_accuracy: 0.6348 - auc_32: 0.8833 - precision_32: 0.9845 - recall_32: 0.5520 - val_loss: 0.5940 - val_categorical_accuracy: 0.6919 - val_auc_32: 0.9332 - val_precision_32: 0.9831 - val_recall_32: 0.6499
Epoch 3/5
Epoch 1/5
13125/13125 - 226s - loss: 0.0947 - categorical_accuracy: 0.6366 - auc_32: 0.8834 - precision_32: 0.9843 - recall_32: 0.5534 - val_loss: 0.5751 - val_categorical_accuracy: 0.6980 - val_auc_32: 0.9366 - val_precision_32: 0.9822 - val_recall_32: 0.6587
Epoch 4/5
Epoch 1/5
13125/13125 - 225s - loss: 0.0947 - categorical_accuracy: 0.6370 - auc_32: 0.8855 - precision_32: 0.9838 - recall_32: 0.5570 - val_loss: 0.5771 - val_categorical_accur

KeyboardInterrupt: ignored